# Content Collection and Processing for TSE-NER Long-tail Entity Extraction

Here we will try to follow the full process required to train and use a NER model.

In [1]:
# This is in case you update modules while working
%load_ext autoreload 
%autoreload 2

## Content Collection (a.k.a. getting a *lot* of papers)

For this step we will be using [sci-paper-miner](https://github.com/ronentk/sci-paper-miner) to download scientific publications from arXiv. Get the code from their repository, and use the following command to run it:

`python crawl_core.py <your-api>` 

In the `crawl_core` file, you can modify the topics and range of years that you want to download, for example here we selected papers only from 2017 for Artificial Intelligence, Computational Complexity, Cryptography and Security, Human-Computer Interaction, Logic in Computer Science, Mathematical Software, Multiagent Systems, Neural and Evolutionary Computing and Sound (in the `crawl_core` file they have all the topics, you can choose what to keep).  In `configs` you can write a name for the folder where the data will be stored.

In [24]:
import json
import os

In [32]:
# This is the path where all the json files were downloaded 

path = 'sci_paper_miner/data/arxiv_2006-2017_cs/raw_query/'

We know that the files we downloaded are in JSON format, but let's check their structure.
For this we can use `os.walk`.

In [41]:
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('.json'):
            print(filename)

144_Computer Science - Artificial Intelligence_2017_5_0.json
144_Computer Science - Artificial Intelligence_2017_5_1.json
144_Computer Science - Artificial Intelligence_2017_5_10.json
144_Computer Science - Artificial Intelligence_2017_5_11.json
144_Computer Science - Artificial Intelligence_2017_5_12.json
144_Computer Science - Artificial Intelligence_2017_5_13.json
144_Computer Science - Artificial Intelligence_2017_5_14.json
144_Computer Science - Artificial Intelligence_2017_5_15.json
144_Computer Science - Artificial Intelligence_2017_5_16.json
144_Computer Science - Artificial Intelligence_2017_5_17.json
144_Computer Science - Artificial Intelligence_2017_5_18.json
144_Computer Science - Artificial Intelligence_2017_5_19.json
144_Computer Science - Artificial Intelligence_2017_5_2.json
144_Computer Science - Artificial Intelligence_2017_5_20.json
144_Computer Science - Artificial Intelligence_2017_5_21.json
144_Computer Science - Artificial Intelligence_2017_5_22.json
144_Compute

It seems like they are not individual articles, so we have to look into one of them.
Let's just take the last one.

In [42]:
json_file = os.path.join(path, filename)
with open(json_file) as f:
    data = json.load(f)

JSON files are loaded as dictionaries:

In [43]:
data.keys()

dict_keys(['authors', 'contributors', 'datePublished', 'description', 'doi', 'downloadUrl', 'fullText', 'fulltextIdentifier', 'id', 'identifiers', 'oai', 'relations', 'repositories', 'subjects', 'title', 'topics', 'types', 'year'])

If we look into one of those keys, it seems like there is many articles, each one with their respective value for the keys above.

In [53]:
data['title'].keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47'])

So we can check the content of the `Title` key, for all articles:

In [61]:
for key in data['title']:
    print(data['title'][key])
    print('-'*50)

A new cosine series antialiasing function and its application to
  aliasing-free glottal source models for speech and singing synthesis
--------------------------------------------------
Investigating the role of musical genre in human perception of music
  stretching resistance
--------------------------------------------------
Enabling Early Audio Event Detection with Neural Networks
--------------------------------------------------
Multi-Speaker Localization Using Convolutional Neural Network Trained
  with Noise
--------------------------------------------------
Acoustic Reflector Localization: Novel Image Source Reversion and Direct
  Localization Methods
--------------------------------------------------
A modulation property of time-frequency derivatives of filtered phase
  and its application to aperiodicity and fo estimation
--------------------------------------------------
Weakly Supervised PLDA Training
--------------------------------------------------
CNN Architectures f

Or the content of all keys, for the first article, which is what we are looking for:

In [60]:
for key in data:
    print(key, data[key]['0'])
    print('-'*50)

authors ['Kawahara, Hideki', 'Sakakibara, Ken-Ichi', 'Banno, Hideki', 'Morise, Masanori', 'Toda, Tomoki', 'Irino, Toshio']
--------------------------------------------------
contributors []
--------------------------------------------------
datePublished 2017-06-08
--------------------------------------------------
description We formulated and implemented a procedure to generate aliasing-free
excitation source signals. It uses a new antialiasing filter in the continuous
time domain followed by an IIR digital filter for response equalization. We
introduced a cosine-series-based general design procedure for the new
antialiasing function. We applied this new procedure to implement the
antialiased Fujisaki-Ljungqvist model. We also applied it to revise our
previous implementation of the antialiased Fant-Liljencrants model. A
combination of these signals and a lattice implementation of the time varying
vocal tract model provides a reliable and flexible basis to test fo extractors
and sourc

Awesome! So we have to iterate for all the downloaded JSON files, for all keys, for all articles, so we can store it in a database. In this case we will use MongoDB.

## Extraction and Storage - MongoDB

We use MongoDB because it is great for prototyping and quick schema changes, but other storage systems can be probably used. Ultimately, all the data is then indexed in Elasticsearch, and that's where we actually query in production.

You can go [here](https://docs.mongodb.com/manual/installation/) to install MongoDB.

Then, using a MongoDB client, create a database, for example _pub_ and a collection, like _publications_.

In [2]:
from pymongo import MongoClient
from pymongo.errors import ServerSelectionTimeoutError
import string

In [3]:
# Default values
mongoDB_IP = '127.0.0.1'
mongoDB_Port = 27017
mongoDB_db = 'pub'

In [4]:
def connect_to_mongo():
    """
    Returns a db connection to the mongo instance
    :return:
    """
    try:
        client = MongoClient(mongoDB_IP, mongoDB_Port)
        db = client[mongoDB_db]
        db.downloads.find_one({'_id': 'test'})
        return db
    except ServerSelectionTimeoutError as e:
        raise Exception("Local MongoDB instance at "+mongoDB_IP+":"+mongoDB_Port+" could not be accessed.") from e

We can test the connection to the database with the following command:

In [5]:
db = connect_to_mongo()

Then we define a function that will take each of the articles from the JSON files downloaded from CORE, and store the information into our collection.

The data obtained from CORE does not separate sections in the text, it is not so important except for the references, since we may use them later on, therefore we also define a function that will manage this.

In [147]:
def arxiv_json_to_mongo(article):
    """
    Creates a new entry in mongodb from the input article
    :return:
    """
    
    mongo_input = {}
    translator = str.maketrans('', '', string.punctuation)
    article_data = article

    mongo_input['title'] = article_data['title']
    mongo_input['authors'] = article_data['authors']
    mongo_input['journal'] = 'arxiv'
    mongo_input['year'] = article_data['year']
    mongo_input['type'] = article_data['subjects']
    mongo_input['content.abstract'] = article_data['description']
    mongo_input['content.keywords'] = article_data['topics']
    mongo_input['content.fulltext'] = article_data['fullText']
    mongo_input['content.references'] = article_data['references']

    mongo_mongo_input = db.publications.update_one(
        {'_id': 'arxiv_' + article_data['id']},
        {'$set': mongo_input},
        upsert=True
    )    
    print('Processed', article_data['id'], 'from JSON')
    
def manage_text_and_refs(article):
    article['fullText'] = article['fullText'].replace('\n', ' ').replace('\r', '')
    if len(article['fullText'].split('References', 1)) == 2:
        article['references'] = article['fullText'].split('References', 1)[1].split('[')
        article['fullText'] = article['fullText'].split('References', 1)[0]
    elif len(article['fullText'].split('REFERENCES', 1)) == 2:
        article['references'] = article['fullText'].split('REFERENCES', 1)[1].split('[')
        article['fullText'] = article['fullText'].split('References', 1)[0]
    else:
        article['fullText'] = article['fullText']
        article['references'] = ''
    return article

Now we have to iterate through all the downloaded JSON files, and all the articles in each file, to process them and introduce them into our database.

In [149]:
for dirpath, dirnames, filenames in os.walk(path):
    # Iterate files...
    for filename in filenames:
        if filename.endswith('.json'):
            json_file = os.path.join(path, filename)
            with open(json_file) as f:
                data = json.load(f)            
                    # Iterate articles in file...
                for article_number in data['title'].keys():
                    article = {}
                    for key in data.keys():
                        article[key] = data[key][str(article_number)]
                        
                    # Process text and references for each article
                    manage_text_and_refs(article)
                    # Store to database
                    arxiv_json_to_mongo(article)

Processed 25036267 from JSON
Processed 2113087 from JSON
Processed 25025591 from JSON
Processed 25015102 from JSON
Processed 24956790 from JSON
Processed 25038409 from JSON
Processed 2246166 from JSON
Processed 73386897 from JSON
Processed 83863968 from JSON
Processed 83867371 from JSON
Processed 94047681 from JSON
Processed 146472595 from JSON
Processed 94063315 from JSON
Processed 29526388 from JSON
Processed 29536914 from JSON
Processed 84090473 from JSON
Processed 84093559 from JSON
Processed 84093915 from JSON
Processed 84093852 from JSON
Processed 84094420 from JSON
Processed 84326602 from JSON
Processed 84328172 from JSON
Processed 84330324 from JSON
Processed 84329889 from JSON
Processed 84326664 from JSON
Processed 84331310 from JSON
Processed 86416235 from JSON
Processed 86419366 from JSON
Processed 86420205 from JSON
Processed 84330603 from JSON
Processed 84330526 from JSON
Processed 84331497 from JSON
Processed 73423992 from JSON
Processed 73417539 from JSON
Processed 73408

Processed 83866609 from JSON
Processed 93946032 from JSON
Processed 93954042 from JSON
Processed 93952440 from JSON
Processed 93874081 from JSON
Processed 93943848 from JSON
Processed 93944339 from JSON
Processed 93944694 from JSON
Processed 93945015 from JSON
Processed 93942643 from JSON
Processed 146474068 from JSON
Processed 146474486 from JSON
Processed 42744287 from JSON
Processed 42715942 from JSON
Processed 42730154 from JSON
Processed 94072332 from JSON
Processed 94071067 from JSON
Processed 94058844 from JSON
Processed 93907363 from JSON
Processed 146475319 from JSON
Processed 84331874 from JSON
Processed 86415709 from JSON
Processed 86415925 from JSON
Processed 86418777 from JSON
Processed 84332442 from JSON
Processed 84330749 from JSON
Processed 84331069 from JSON
Processed 84330624 from JSON
Processed 84330701 from JSON
Processed 84331502 from JSON
Processed 84331595 from JSON
Processed 86415305 from JSON
Processed 86414737 from JSON
Processed 86417914 from JSON
Processed 7

Processed 93944988 from JSON
Processed 93947576 from JSON
Processed 93948719 from JSON
Processed 93951313 from JSON
Processed 93953333 from JSON
Processed 93953566 from JSON
Processed 93936930 from JSON
Processed 93936512 from JSON
Processed 93940234 from JSON
Processed 93869650 from JSON
Processed 93907518 from JSON
Processed 93942191 from JSON
Processed 93945152 from JSON
Processed 94032693 from JSON
Processed 146471726 from JSON
Processed 94024498 from JSON
Processed 129246622 from JSON
Processed 129264059 from JSON
Processed 129225652 from JSON
Processed 141534671 from JSON
Processed 141535333 from JSON
Processed 141534796 from JSON
Processed 93883528 from JSON
Processed 83845686 from JSON
Processed 83848072 from JSON
Processed 83849835 from JSON
Processed 83852895 from JSON
Processed 83859106 from JSON
Processed 83864340 from JSON
Processed 83856208 from JSON
Processed 83858660 from JSON
Processed 73383291 from JSON
Processed 83869383 from JSON
Processed 83841305 from JSON
Process

Processed 86421012 from JSON
Processed 86419000 from JSON
Processed 86417477 from JSON
Processed 86417617 from JSON
Processed 86419961 from JSON
Processed 29518418 from JSON
Processed 83831911 from JSON
Processed 83834080 from JSON
Processed 83833917 from JSON
Processed 83834840 from JSON
Processed 83832976 from JSON
Processed 83833715 from JSON
Processed 146475980 from JSON
Processed 146473775 from JSON
Processed 94062508 from JSON
Processed 94050938 from JSON
Processed 146474949 from JSON
Processed 94070006 from JSON
Processed 94075467 from JSON
Processed 94074604 from JSON
Processed 129357551 from JSON
Processed 93940465 from JSON
Processed 83836132 from JSON
Processed 83835487 from JSON
Processed 83835069 from JSON
Processed 42669201 from JSON
Processed 42706142 from JSON
Processed 42719258 from JSON
Processed 74203909 from JSON
Processed 74251340 from JSON
Processed 42743024 from JSON
Processed 74251078 from JSON
Processed 73960926 from JSON
Processed 73960988 from JSON
Processed 

Processed 42714190 from JSON
Processed 29509195 from JSON
Processed 84094933 from JSON
Processed 73442277 from JSON
Processed 86414492 from JSON
Processed 42641564 from JSON
Processed 42688561 from JSON
Processed 42655868 from JSON
Processed 42742913 from JSON
Processed 74202836 from JSON
Processed 74203978 from JSON
Processed 74203947 from JSON
Processed 42743606 from JSON
Processed 129239884 from JSON
Processed 129243931 from JSON
Processed 73993085 from JSON
Processed 42730879 from JSON
Processed 42739232 from JSON
Processed 42737724 from JSON
Processed 129270085 from JSON
Processed 129269247 from JSON
Processed 129274513 from JSON
Processed 94031902 from JSON
Processed 141529941 from JSON
Processed 141530082 from JSON
Processed 74250873 from JSON
Processed 74251720 from JSON
Processed 78509678 from JSON
Processed 141533043 from JSON
Processed 141532723 from JSON
Processed 78512366 from JSON
Processed 146472859 from JSON
Processed 94046603 from JSON
Processed 93908494 from JSON
Proc

...

Processed 94025848 from JSON
Processed 74251795 from JSON
Processed 74251252 from JSON
Processed 146473522 from JSON
Processed 146472257 from JSON
Processed 94056134 from JSON
Processed 129354224 from JSON
Processed 129357494 from JSON
Processed 129363237 from JSON
Processed 141532659 from JSON
Processed 93907882 from JSON
Processed 93915955 from JSON
Processed 93947042 from JSON
Processed 93946830 from JSON
Processed 93943824 from JSON
Processed 129361680 from JSON
Processed 129224567 from JSON
Processed 129355676 from JSON
Processed 93886529 from JSON
Processed 129203131 from JSON
Processed 129238032 from JSON
Processed 78510782 from JSON
Processed 73387570 from JSON
Processed 73357496 from JSON
Processed 83846492 from JSON
Processed 83866522 from JSON
Processed 83869172 from JSON
Processed 73378699 from JSON
Processed 73355678 from JSON
Processed 83841358 from JSON
Processed 83833425 from JSON
Processed 83836277 from JSON
Processed 73353766 from JSON
Processed 83845615 from JSON
Pro

Processed 83860964 from JSON
Processed 83859582 from JSON
Processed 83861563 from JSON
Processed 83862270 from JSON
Processed 83855807 from JSON
Processed 83851160 from JSON
Processed 83849848 from JSON
Processed 83863862 from JSON
Processed 73360346 from JSON
Processed 83849088 from JSON
Processed 83847981 from JSON
Processed 93877417 from JSON
Processed 83866499 from JSON
Processed 83832849 from JSON
Processed 83838241 from JSON
Processed 83839167 from JSON
Processed 83853275 from JSON
Processed 73393242 from JSON
Processed 73361148 from JSON
Processed 83847160 from JSON
Processed 73420526 from JSON
Processed 73407389 from JSON
Processed 73408623 from JSON
Processed 84092329 from JSON
Processed 84092855 from JSON
Processed 73420728 from JSON
Processed 84094969 from JSON
Processed 73953774 from JSON
Processed 84330096 from JSON
Processed 84327283 from JSON
Processed 42680346 from JSON
Processed 73992924 from JSON
Processed 73987595 from JSON
Processed 84333196 from JSON
Processed 8432

Processed 83833450 from JSON
Processed 94034127 from JSON
Processed 94075171 from JSON
Processed 129353838 from JSON
Processed 93940465 from JSON
Processed 83840614 from JSON
Processed 74251340 from JSON
Processed 42698180 from JSON
Processed 73958868 from JSON
Processed 84330224 from JSON
Processed 84331955 from JSON
Processed 84331272 from JSON
Processed 73990846 from JSON
Processed 73993061 from JSON
Processed 42728177 from JSON
Processed 42736156 from JSON
Processed 93908285 from JSON
Processed 93907530 from JSON
Processed 93912161 from JSON
Processed 93867227 from JSON
Processed 78508000 from JSON
Processed 93946665 from JSON
Processed 83860612 from JSON
Processed 83852880 from JSON
Processed 83853061 from JSON
Processed 83852789 from JSON
Processed 83855500 from JSON
Processed 83847817 from JSON
Processed 83850365 from JSON
Processed 83855719 from JSON
Processed 83842711 from JSON
Processed 129233569 from JSON
Processed 83865840 from JSON
Processed 83870426 from JSON
Processed 83

Processed 94047681 from JSON
Processed 84094468 from JSON
Processed 84093340 from JSON
Processed 86421626 from JSON
Processed 84326431 from JSON
Processed 86415559 from JSON
Processed 84329252 from JSON
Processed 73414888 from JSON
Processed 73405938 from JSON
Processed 78507488 from JSON
Processed 141531952 from JSON
Processed 141532753 from JSON
Processed 93891203 from JSON
Processed 93936890 from JSON
Processed 93887776 from JSON
Processed 93942772 from JSON
Processed 42638983 from JSON
Processed 83839161 from JSON
Processed 73384954 from JSON
Processed 73394009 from JSON
Processed 83857668 from JSON
Processed 83865306 from JSON
Processed 83846248 from JSON
Processed 83850977 from JSON
Processed 73377839 from JSON
Processed 83864132 from JSON
Processed 94078807 from JSON
Processed 129358344 from JSON
Processed 129361018 from JSON
Processed 129243468 from JSON
Processed 73961238 from JSON
Processed 83838221 from JSON
Processed 83847342 from JSON
Processed 86419132 from JSON
Processed

Processed 86420799 from JSON
Processed 129240605 from JSON
Processed 141531716 from JSON
Processed 93900853 from JSON
Processed 129356830 from JSON
Processed 93949731 from JSON
Processed 93948310 from JSON
Processed 83835194 from JSON
Processed 146474357 from JSON
Processed 73374050 from JSON
Processed 93937091 from JSON
Processed 146471442 from JSON
Processed 94041308 from JSON
Processed 94049681 from JSON
Processed 146473431 from JSON
Processed 83867553 from JSON
Processed 73384129 from JSON
Processed 83848680 from JSON
Processed 78509187 from JSON
Processed 141538825 from JSON
Processed 73441981 from JSON
Processed 84093274 from JSON
Processed 93955721 from JSON
Processed 83831680 from JSON
Processed 93888084 from JSON
Processed 129235802 from JSON
Processed 86422136 from JSON
Processed 84331601 from JSON
Processed 86415268 from JSON
Processed 146471597 from JSON
Processed 141534242 from JSON
Processed 141538453 from JSON
Processed 141532549 from JSON
Processed 83844191 from JSON
Pr

Done! We have a bunch of files in our database! (**8848** articles in this demo)

## Indexing - Elasticsearch 

We use Elasticsearch for the quick and very flexible (elastic?) queries across the full text of articles, so we have to index all the content from the database for this.

Once again, we have to [install](https://www.elastic.co/guide/en/elasticsearch/reference/current/_installation.html) it and run the service...yay!

Once it's running we can connect and check status:

In [6]:
import pymongo
import elasticsearch
import requests
import nltk
from elasticsearch import helpers
import re
import string

In [7]:
client = pymongo.MongoClient('localhost:' + str(mongoDB_Port))
publications_collection = client.pub.publications
es = elasticsearch.Elasticsearch([{'host': 'localhost', 'port': 9200}],
                                 timeout=30, max_retries=10, retry_on_timeout=True)

In [7]:
es.cluster.health()

{'active_primary_shards': 10,
 'active_shards': 10,
 'active_shards_percent_as_number': 50.0,
 'cluster_name': 'elasticsearch',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 1,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 1,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'yellow',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 10}

### Full text indexing

First we index the full content of every article in the database.

(This step corresponds to `collection_extraction -> index_papersfulltext.py` in the code)

In [8]:
def extract_metadata(documents):
    list_of_docs = []
    for i, r in enumerate(documents):
        extracted = {
                "_id": "",
                "title": "",
                "publication": "",
                "year": "",
                "content": "",
                "abstract": "",
                "authors": [],
                "references": []}
        extracted['_id'] = r['_id']
        extracted['title'] = r['title']
        extracted['publication'] = r['journal']
        extracted['year'] = r['year']
        extracted['content'] = r['content']['fulltext']
        extracted['abstract'] = r['content']['abstract']
        extracted['authors'] = r['authors']
        extracted['references'] = r['content']['references']
        list_of_docs.append(extracted)
    return list_of_docs

In [26]:
filter_publications = ['arxiv'] # Here we could also put PubMed or other sources

extracted_publications = []
for publication in filter_publications:
    query ={'$and': [{'journal': publication}, {'content.fulltext': {'$exists': True}}]}                   
    results = publications_collection.find(query)
    extracted_publications.append(extract_metadata(results))

In [166]:
for publication in extracted_publications:
    actions = []
    for article in publication:
        authors = []
        if len(article['authors']) > 0:
            if type(article['authors'][0]) == list:
                try:
                    for name in article['authors']:
                        authors.append(', '.join([name[1], name[0]]))
                    authors = list(set(authors))
                except:
                    pass
            else:
                authors = article['authors']
        actions.append({
            "_index": "ir", 
            "_type": "publications",  
            "_id": article['_id'],
            "_source": {
                "title": article["title"],
                "journal": article['publication'],
                "year": str(article['year']),
                "content": article["content"],
                "abstract": article["abstract"],
                "authors": authors,
                "references": article["references"]
            }
        })
    if len(actions) == 0:
        continue
    res = helpers.bulk(es, actions)
    print('Done with', res, 'articles added to index')

Done with (8848, []) articles added to index


We can look for anything in the text, like this:

In [180]:
res = es.search(index = "ir", body = {"query": {"match": {"title" : "netflix"}}}, size = 10)

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit['_id'], hit['_source']['title'])
    print(hit['_id'], hit['_source']['abstract'])
    print('-'*50)

Got 1 Hits:
arxiv_84094279 Re-Evaluating the Netflix Prize - Human Uncertainty and its Impact on
  Reliability
arxiv_84094279 In this paper, we examine the statistical soundness of comparative
assessments within the field of recommender systems in terms of reliability and
human uncertainty. From a controlled experiment, we get the insight that users
provide different ratings on same items when repeatedly asked. This volatility
of user ratings justifies the assumption of using probability densities instead
of single rating scores. As a consequence, the well-known accuracy metrics
(e.g. MAE, MSE, RMSE) yield a density themselves that emerges from convolution
of all rating densities. When two different systems produce different RMSE
distributions with significant intersection, then there exists a probability of
error for each possible ranking. As an application, we examine possible ranking
errors of the Netflix Prize. We are able to show that all top rankings are more
or less subject to h

### Sentence Indexing 
For the sentence expansion step of our process, we need to have sentences indexed. This is because we need to find similar sentences as well as their surrounding sentences, for context. 

In addition, we create a file with all the text, of all our articles, and we will use it later for the training of embedding models.

(This step corresponds to `collection_extraction -> index_twosent.py` in the code)

In [8]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [203]:
for publication in extracted_publications:
    for article in publication:
        actions = []
        cleaned = []
        dataset_sent = []
        other_sent = []

        lines = (sent_detector.tokenize(article['content'].strip()))
        
        # This will be used for the training of word2vec and doc2vec
        # You need to create the data folder beforehand
        with open('data/full_text_corpus.txt', 'a', encoding='utf-8') as f:
            for line in lines:
                f.write(line)
        f.close()
        
        if len(lines) < 3:
            continue

        for i in range(len(lines)):
            words = nltk.word_tokenize(lines[i])
            word_lengths = [len(x) for x in words]
            average_word_length = sum(word_lengths) / len(word_lengths)
            if average_word_length < 4:
                continue

            two_sentences = ''
            try:
                two_sentences = lines[i] + ' ' + lines[i - 1]
            except:
                two_sentences = lines[i] + ' ' + lines[i + 1]

            dataset_sent.append(two_sentences)

        for num, added_lines in enumerate(dataset_sent):
            actions.append({
                "_index": "twosent",
                "_type": "twosentnorules",
                "_id": article['_id'] + str(num),
                "_source": {
                    "title": article['title'],
                    "content.chapter.sentpositive": added_lines,
                    "paper_id": article['_id']
                }})

        if len(actions) == 0:
            continue
        res = helpers.bulk(es, actions)
    print('Done')

Done


In [193]:
res = es.search(index = "twosent", body = {"query": {"match": {"content.chapter.sentpositive" : "netflix"}}}, size = 10)

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit['_id'], hit['_source']['title'])
    print('-'*50)

Got 174 Hits:
arxiv_14153268986 Learning to Rank based on Analogical Reasoning
--------------------------------------------------
arxiv_146472211389 One-sided Differential Privacy
--------------------------------------------------
arxiv_129348860158 Differentially Private Query Learning: from Data Publishing to Model
  Publishing
--------------------------------------------------
arxiv_14153268985 Learning to Rank based on Analogical Reasoning
--------------------------------------------------
arxiv_129348860154 Differentially Private Query Learning: from Data Publishing to Model
  Publishing
--------------------------------------------------
arxiv_129348860153 Differentially Private Query Learning: from Data Publishing to Model
  Publishing
--------------------------------------------------
arxiv_93942601272 Interacting Attention-gated Recurrent Networks for Recommendation
--------------------------------------------------
arxiv_129348860157 Differentially Private Query Learning: from

### Doc2vec Indexing

Once again, we index the sentences from the full text file created before, which will be used for the Sentence Expansion using doc2vec.

(This step corresponds to `collection_extraction -> index_doc2vec.py` in the code)

In [9]:
file = open('data/full_text_corpus.txt', 'r', encoding='utf-8')
text = file.read()
file.close()
sentences = nltk.tokenize.sent_tokenize(text)
print('Sentences ready')
count = 0
docLabels = []
actions = []

for i, sent in enumerate(sentences):
    try:
        neighbors = sentences[i + 1]
        neighbor_count = count + 1
    except:
        neighbors = sentences[i - 1]
        neighbor_count = count - 1

    docLabels.append(count)
    actions.append({
        "_index": "devtwosentnew",
        "_type": "devtwosentnorulesnew",
        "_id": count,
        "_source": {
            "content.chapter.sentpositive": sent,
            "content.chapter.sentnegtive": neighbors,
            "neighborcount": neighbor_count
        }})
    count = count + 1

print(len(sentences))
print(len(docLabels))
res = helpers.bulk(es, actions)
print(res)

Sentences ready
248840
248840
(248840, [])


## Training Embedding Models (word2vec & doc2vec)

For the Term and Sentence expansion, we need to find similar words or sentences, and we use word2vec and doc2vec respectively. We have to generate a document with all the words in our corpus and one with all the sentences, both will be stored in the data folder. While this is not a full implementation, we have around 4 million sentences, so it is a considerable example, and processing can take some time.

(This step corresponds to `data_preparation -> prepare_embedding_data.py` in the code)

In [27]:
papers_text = []
sentence_text = []
translator = str.maketrans('', '', string.punctuation)

for publication in extracted_publications:
    for article in publication:
        query = {"query":
                     {"match":
                          {"_id":
                               {"query": article['_id'],
                                "operator": "and"
                                }
                           }
                      }
                 }

        results = es.search(index="ir", body=query, size=200)
        for doc in results['hits']['hits']:
            fulltext = doc["_source"]["content"]
            fulltext = re.sub("[\[].*?[\]]", "", fulltext)
            cleaned_text = fulltext.translate(translator)
            papers_text.append(cleaned_text.lower())
            sentence_text.append(fulltext)
    print('Done', '-' * 100)

tokens = " ".join(papers_text)

f = open("data/dataWord2vec.txt", "w", encoding='utf-8')
f.write(tokens)
f.close()

f = open("data/dataDoc2vec.txt", "w", encoding='utf-8')
for line in sentence_text:
    f.write(line)
f.close()

Done ----------------------------------------------------------------------------------------------------


Before training, verify that you have the optimized version of Gensim using Cython, for better performance.

In [ ]:
import gensim
assert gensim.models.doc2vec.FAST_VERSION > -1

Now that we have both documents with all the text, we can use them for the training. 

We will use a bigram model for word2vec, since it's common that entities are represented as two words. You can run the command from your Python console or directly here, using the magic `%run` syntax. The structure of this command is as follows:

`%run script_to_run training_data model_output word_vector_output`

This was run in a normal Lenovo machine with 8GB and i7 processor. As you can see by the timestamps below, it might take a while...

In [52]:
%run data_preparation/train_word2vec.py data/dataDoc2vec.txt data/modelword2vecbigram.model data/modelword2vecbigram.vec

2018-05-14 17:07:47,086: INFO: running data_preparation/train_word2vec.py data/dataDoc2vec.txt data/modelword2vecbigram.model data/modelword2vecbigram.vec
2018-05-14 17:28:19,989: INFO: collecting all words and their counts
2018-05-14 17:28:20,025: INFO: PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-05-14 17:28:20,610: INFO: PROGRESS: at sentence #10000, processed 229355 words and 118344 word types
2018-05-14 17:28:21,168: INFO: PROGRESS: at sentence #20000, processed 454354 words and 192778 word types
2018-05-14 17:28:21,542: INFO: PROGRESS: at sentence #30000, processed 623029 words and 227709 word types
2018-05-14 17:28:22,066: INFO: PROGRESS: at sentence #40000, processed 841989 words and 295848 word types
2018-05-14 17:28:22,678: INFO: PROGRESS: at sentence #50000, processed 1084244 words and 360848 word types
2018-05-14 17:28:23,274: INFO: PROGRESS: at sentence #60000, processed 1344739 words and 431377 word types
2018-05-14 17:28:23,827: INFO: PROGRESS: at se

2018-05-14 17:29:07,734: INFO: PROGRESS: at sentence #740000, processed 17361048 words and 3022251 word types
2018-05-14 17:29:08,990: INFO: PROGRESS: at sentence #750000, processed 17594845 words and 3042351 word types
2018-05-14 17:29:10,103: INFO: PROGRESS: at sentence #760000, processed 17814742 words and 3074467 word types
2018-05-14 17:29:10,995: INFO: PROGRESS: at sentence #770000, processed 18025720 words and 3091216 word types
2018-05-14 17:29:12,946: INFO: PROGRESS: at sentence #780000, processed 18283854 words and 3116093 word types
2018-05-14 17:29:14,310: INFO: PROGRESS: at sentence #790000, processed 18531023 words and 3154856 word types
2018-05-14 17:29:15,352: INFO: PROGRESS: at sentence #800000, processed 18774891 words and 3177816 word types
2018-05-14 17:29:16,261: INFO: PROGRESS: at sentence #810000, processed 19014899 words and 3198612 word types
2018-05-14 17:29:17,139: INFO: PROGRESS: at sentence #820000, processed 19255256 words and 3219610 word types
2018-05-14

2018-05-14 17:30:01,067: INFO: PROGRESS: at sentence #1490000, processed 34208850 words and 4778377 word types
2018-05-14 17:30:01,721: INFO: PROGRESS: at sentence #1500000, processed 34450358 words and 4787806 word types
2018-05-14 17:30:02,259: INFO: PROGRESS: at sentence #1510000, processed 34639208 words and 4809380 word types
2018-05-14 17:30:02,807: INFO: PROGRESS: at sentence #1520000, processed 34825756 words and 4826088 word types
2018-05-14 17:30:03,374: INFO: PROGRESS: at sentence #1530000, processed 35024839 words and 4839282 word types
2018-05-14 17:30:04,114: INFO: PROGRESS: at sentence #1540000, processed 35251130 words and 4855824 word types
2018-05-14 17:30:04,662: INFO: PROGRESS: at sentence #1550000, processed 35438098 words and 4866389 word types
2018-05-14 17:30:06,181: INFO: PROGRESS: at sentence #1560000, processed 35667352 words and 4889372 word types
2018-05-14 17:30:08,014: INFO: PROGRESS: at sentence #1570000, processed 35900369 words and 4911339 word types
2

2018-05-14 17:31:01,236: INFO: PROGRESS: at sentence #2230000, processed 50414210 words and 6402990 word types
2018-05-14 17:31:02,063: INFO: PROGRESS: at sentence #2240000, processed 50674430 words and 6429017 word types
2018-05-14 17:31:02,681: INFO: PROGRESS: at sentence #2250000, processed 50893583 words and 6449521 word types
2018-05-14 17:31:03,397: INFO: PROGRESS: at sentence #2260000, processed 51129908 words and 6475182 word types
2018-05-14 17:31:04,102: INFO: PROGRESS: at sentence #2270000, processed 51368727 words and 6502858 word types
2018-05-14 17:31:04,802: INFO: PROGRESS: at sentence #2280000, processed 51600670 words and 6533322 word types
2018-05-14 17:31:05,461: INFO: PROGRESS: at sentence #2290000, processed 51828992 words and 6560843 word types
2018-05-14 17:31:06,074: INFO: PROGRESS: at sentence #2300000, processed 52036768 words and 6589634 word types
2018-05-14 17:31:06,748: INFO: PROGRESS: at sentence #2310000, processed 52275567 words and 6609790 word types
2

2018-05-14 17:31:53,454: INFO: PROGRESS: at sentence #2970000, processed 66946087 words and 8174950 word types
2018-05-14 17:31:54,415: INFO: PROGRESS: at sentence #2980000, processed 67214269 words and 8206242 word types
2018-05-14 17:31:55,145: INFO: PROGRESS: at sentence #2990000, processed 67446789 words and 8230690 word types
2018-05-14 17:31:56,513: INFO: PROGRESS: at sentence #3000000, processed 67700484 words and 8253589 word types
2018-05-14 17:31:57,514: INFO: PROGRESS: at sentence #3010000, processed 67911983 words and 8265359 word types
2018-05-14 17:31:58,784: INFO: PROGRESS: at sentence #3020000, processed 68136781 words and 8283506 word types
2018-05-14 17:32:00,477: INFO: PROGRESS: at sentence #3030000, processed 68369668 words and 8305425 word types
2018-05-14 17:32:01,625: INFO: PROGRESS: at sentence #3040000, processed 68605304 words and 8327038 word types
2018-05-14 17:32:02,570: INFO: PROGRESS: at sentence #3050000, processed 68844240 words and 8343521 word types
2

...

2018-05-14 18:25:08,807: INFO: EPOCH 5 - PROGRESS: at 29.46% examples, 121608 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:25:09,837: INFO: EPOCH 5 - PROGRESS: at 29.70% examples, 121673 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:25:10,870: INFO: EPOCH 5 - PROGRESS: at 29.92% examples, 121698 words/s, in_qsize 1, out_qsize 0
2018-05-14 18:25:11,879: INFO: EPOCH 5 - PROGRESS: at 30.17% examples, 121677 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:25:12,909: INFO: EPOCH 5 - PROGRESS: at 30.41% examples, 121702 words/s, in_qsize 1, out_qsize 0
2018-05-14 18:25:13,974: INFO: EPOCH 5 - PROGRESS: at 30.66% examples, 121733 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:25:14,993: INFO: EPOCH 5 - PROGRESS: at 30.93% examples, 121743 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:25:16,010: INFO: EPOCH 5 - PROGRESS: at 31.19% examples, 121765 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:25:17,044: INFO: EPOCH 5 - PROGRESS: at 31.43% examples, 121771 words/s, in_qsize 0, out_qsize 0
2

2018-05-14 18:28:16,609: INFO: EPOCH 5 - PROGRESS: at 47.17% examples, 80019 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:28:17,616: INFO: EPOCH 5 - PROGRESS: at 47.44% examples, 80170 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:28:18,653: INFO: EPOCH 5 - PROGRESS: at 47.71% examples, 80332 words/s, in_qsize 1, out_qsize 0
2018-05-14 18:28:19,655: INFO: EPOCH 5 - PROGRESS: at 47.94% examples, 80487 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:28:20,667: INFO: EPOCH 5 - PROGRESS: at 48.19% examples, 80636 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:28:21,716: INFO: EPOCH 5 - PROGRESS: at 48.43% examples, 80797 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:28:22,742: INFO: EPOCH 5 - PROGRESS: at 48.70% examples, 80942 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:28:23,772: INFO: EPOCH 5 - PROGRESS: at 48.92% examples, 81106 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:28:24,802: INFO: EPOCH 5 - PROGRESS: at 49.14% examples, 81268 words/s, in_qsize 0, out_qsize 0
2018-05-14

2018-05-14 18:29:34,011: INFO: EPOCH 5 - PROGRESS: at 65.88% examples, 89850 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:29:35,074: INFO: EPOCH 5 - PROGRESS: at 66.13% examples, 89977 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:29:36,095: INFO: EPOCH 5 - PROGRESS: at 66.38% examples, 90092 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:29:37,101: INFO: EPOCH 5 - PROGRESS: at 66.64% examples, 90203 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:29:38,107: INFO: EPOCH 5 - PROGRESS: at 66.86% examples, 90296 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:29:39,146: INFO: EPOCH 5 - PROGRESS: at 67.08% examples, 90414 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:29:40,196: INFO: EPOCH 5 - PROGRESS: at 67.32% examples, 90515 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:29:41,236: INFO: EPOCH 5 - PROGRESS: at 67.60% examples, 90615 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:29:42,304: INFO: EPOCH 5 - PROGRESS: at 67.85% examples, 90726 words/s, in_qsize 0, out_qsize 0
2018-05-14

2018-05-14 18:30:51,313: INFO: EPOCH 5 - PROGRESS: at 83.84% examples, 96816 words/s, in_qsize 1, out_qsize 0
2018-05-14 18:30:52,358: INFO: EPOCH 5 - PROGRESS: at 84.07% examples, 96888 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:30:53,366: INFO: EPOCH 5 - PROGRESS: at 84.34% examples, 96968 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:30:54,370: INFO: EPOCH 5 - PROGRESS: at 84.61% examples, 97035 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:30:55,385: INFO: EPOCH 5 - PROGRESS: at 84.84% examples, 97113 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:30:56,399: INFO: EPOCH 5 - PROGRESS: at 85.08% examples, 97176 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:30:57,458: INFO: EPOCH 5 - PROGRESS: at 85.32% examples, 97250 words/s, in_qsize 0, out_qsize 0
2018-05-14 18:30:58,474: INFO: EPOCH 5 - PROGRESS: at 85.53% examples, 97313 words/s, in_qsize 1, out_qsize 0
2018-05-14 18:30:59,508: INFO: EPOCH 5 - PROGRESS: at 85.75% examples, 97345 words/s, in_qsize 0, out_qsize 0
2018-05-14

2018-05-14 18:32:00,588: INFO: EPOCH - 5 : training on 76563714 raw words (55176038 effective words) took 550.0s, 100326 effective words/s
2018-05-14 18:32:00,589: INFO: training on a 382818570 raw words (275877426 effective words) took 2718.8s, 101471 effective words/s
2018-05-14 18:32:00,597: INFO: saving Word2Vec object under data/modelword2vecbigram.model, separately None
2018-05-14 18:32:00,604: INFO: storing np array 'vectors' to data/modelword2vecbigram.model.wv.vectors.npy
2018-05-14 18:32:02,513: INFO: not storing attribute vectors_norm
2018-05-14 18:32:02,518: INFO: storing np array 'syn1neg' to data/modelword2vecbigram.model.trainables.syn1neg.npy
2018-05-14 18:32:04,097: INFO: not storing attribute cum_table
2018-05-14 18:32:08,458: INFO: saved data/modelword2vecbigram.model
2018-05-14 18:32:08,461: INFO: storing 1169943x100 projection weights into data/modelword2vecbigram.vec


In [53]:
%run train_doc2vec.py data/dataDoc2vec.txt data/doc2vec.model

2018-05-14 18:35:37,494: ERROR: File `'train_doc2vec.py'` not found.
